TODO
- genomic data needs to be PCA and split up into train/val/data
- get rid of acc

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

# can revome the lines that need these
import os
import pickle

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import utils

from datasets import GeneralDataset
# import Transforms as myTransforms
from Transforms import get_transformations
from utils import get_data_splits, get_input_params

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Choose parameters
- 4 channel full resolution:
```python
    task \in ['idh', '1p19q']
    dataformat \in ['raw3D', 'crop3Dslice', 'modality3D']
    modality \in ['t1ce', 'flair', 't2', 't1', 't1ce-t1']
    mtl \in [True, False]
    include_genomic_data \in [True, False]
```

In [3]:
task = 'idh'
dataformat = 'crop3Dslice'
modality = 't1ce' # only relevent for 'modality3D' dataformat
mtl = True
include_genomic_data = False # don't include genomic data

null_genomic = not include_genomic_data

dataformat, channels, resize_shape = get_input_params(dataformat)

In [4]:
print('task:\t\t', task)
print('mtl:\t\t', mtl)
print('dataformat:\t', dataformat)
print('channels:\t', channels)
print('modality:\t', modality)
print('resize_shape:\t', resize_shape)
print('null_genomic:\t', null_genomic)

task:		 idh
mtl:		 True
dataformat:	 mtl_cropped
channels:	 4
modality:	 t1ce
resize_shape:	 (64, 64, 64)
null_genomic:	 True


In [5]:
# MRI directory
image_dir = '../data/all_brats_scans/'

# metadata for all brats (including tcia) data
best_model_loc = '../pretrained/espnet_3d_brats.pth' # segmentation model weights
glioma_metadata_df = pd.read_csv('../data/glioma_metadata.csv', index_col=0) # metadata file
glioma_metadata_df.loc[['Brats18_TCIA09_462_1', 'Brats18_TCIA10_236_1'], 'idh'] = 1 ######

# subtype_df = pd.read_csv('../../tcga_data_cleaning/data/processed_data/santa_cruz/patient_metadata/subtype_spreadsheet_1128-36.csv', index_col=0)
# non_mol_gbm_idxs = subtype_df.loc[subtype_df['mol_gbm'] == -1].index
# non_mol_df = glioma_metadata_df.loc[(glioma_metadata_df['tciaID'].isin(non_mol_gbm_idxs))
#                       & (glioma_metadata_df['idh'] == 0)]
# val_non_mol_idxs = non_mol_df.loc[non_mol_df['phase'] == 'val'].index
# glioma_metadata_df.loc[val_non_mol_idxs, 'idh'] = -1


# get training splits
train_df, val_df, classes = get_data_splits(metadata_df=glioma_metadata_df, task=task, mtl=mtl)

# map between brats dataset and tcia data (tcia data is avalible for a subset of the brats patients)
brats2tcia_df = glioma_metadata_df['tciaID']
# brats2tcia_df = pd.read_csv('../../miccai_clean/data/brats2tcia_df_542x1.csv', index_col=0)

# these are labeled files (they were paths in old dataloader) but they are dataframes
labels_dict = {'train':train_df, 'val':val_df, 'data':glioma_metadata_df}

genomic_data_dict = {'train':'../data/MGL/MGL_235x50.csv', 'val':'../data/MGL/MGL_235x50.csv'}

label = task

print('Train size', len(train_df))

Train size 467


In [6]:
# glioma_metadata_df = pd.read_csv('../data/glioma_metadata.csv', index_col=0) 
# glioma_metadata_df

In [7]:
# subtype_df = pd.read_csv('../../tcga_data_cleaning/data/processed_data/santa_cruz/patient_metadata/subtype_spreadsheet_1128-36.csv', index_col=0)

In [8]:
# non_mol_gbm_idxs = subtype_df.loc[subtype_df['mol_gbm'] == -1].index

In [9]:
# non_mol_df = glioma_metadata_df.loc[(glioma_metadata_df['tciaID'].isin(non_mol_gbm_idxs))
#                       & (glioma_metadata_df['idh'] == 0)]
# val_non_mol_idxs = non_mol_df.loc[non_mol_df['phase'] == 'val'].index

In [10]:
# val_non_mol_idxs

# 3D cropped

In [11]:
# batch size
train_transformations, seg_transformations, val_transformations = get_transformations(channels=channels, 
                                                                                      resize_shape=resize_shape, 
                                                                                      prob_voxel_zero=0.2, 
                                                                                      prob_true=0.8, 
                                                                                      prob_channel_zero=0.5,
                                                                                      mtl=False)
data_transforms = {'train': train_transformations, 'val':   val_transformations}
                   
transformed_dataset_train = GeneralDataset(csv_file=train_df, ## change this from "csv_file"
                                           root_dir=image_dir,
                                           genomic_csv_file = genomic_data_dict['train'],
                                           transform=train_transformations,
                                           seg_transform=seg_transformations,
                                           label=label,
                                           classes=classes,
                                           dataformat=dataformat,
                                           returndims=resize_shape,
                                           brats2tcia_df=brats2tcia_df,
                                           null_genomic = null_genomic,
                                           pretrained=best_model_loc,
                                           modality=modality)

transformed_dataset_val = GeneralDataset(csv_file=val_df,
                                         root_dir=image_dir,
                                         genomic_csv_file = genomic_data_dict['val'],
                                         transform=val_transformations,
                                         seg_transform=seg_transformations,
                                         label=label,
                                         classes=classes,
                                         dataformat=dataformat,
                                         returndims=resize_shape,
                                         brats2tcia_df=brats2tcia_df,
                                         null_genomic = null_genomic,
                                         pretrained=best_model_loc,
                                         modality=modality)


image_datasets = {'train':transformed_dataset_train, 'val':transformed_dataset_val}

train_batch_size, val_batch_size = 4, 4
dataloader_train = DataLoader(image_datasets['train'], batch_size=train_batch_size, shuffle=True, num_workers=4)
dataloader_val = DataLoader(image_datasets['val'], batch_size=val_batch_size, shuffle=True, num_workers=4)

dataloaders = {'train':dataloader_train, 'val':dataloader_val}
dataset_sizes = {'train':len(image_datasets['train']), 'val':len(image_datasets['val'])}

class_names = image_datasets['train'].classes
class_names

['wildtype', 'mutant']

In [12]:
# # visulize training (or validation) data
# for i, data in enumerate(dataloaders['train']):
#     # data batch
#     (image, seg_image, genomic_data, seg_probs), label, (OS, OS_EVENT), bratsID = data
#     # print scan ID
#     print(bratsID[0])
    
#     # format MRI images (slices of volumetric input)
#     img = image[0,:, :, :, int(image.shape[-1]/2)].squeeze()
#     img = utils.make_grid(img)
#     img = img.detach().cpu().numpy()
    
#     # plot images
#     plt.figure(figsize=(15, 8))
#     img_list = [img[i].T for i in range(channels)] # 1 image per channel
#     plt.imshow(np.hstack(img_list), cmap='Greys_r')
#     plt.show()

#     ## plot segmentation mask ##
#     seg_img = seg_image[0, :, :, :, int(seg_image.shape[-1]/2)].squeeze()
#     seg_img = utils.make_grid(seg_img).detach().cpu().numpy()

#     plt.figure(figsize=(4, 4))
#     plt.imshow(np.hstack([seg_img[0].T]), cmap='Greys_r')
#     plt.show()

#     break

In [13]:
from torch.utils.tensorboard import SummaryWriter
img_dims = str(resize_shape[0]) + 'x' + str(resize_shape[1]) + 'x' + str(resize_shape[2])
model_outfile_dir = '3D_' + task + '_' + modality+'_mtl-' + str(mtl) + '_' \
                    + img_dims + '_genomic-' + str(include_genomic_data)
print('tensorboad:', model_outfile_dir)
writer = SummaryWriter('runs1/'+model_outfile_dir)

tensorboad: 3D_idh_t1ce_mtl-True_64x64x64_genomic-False


In [14]:
# # Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

In [15]:
num_classes = 2

label_df = train_df.loc[train_df[task].isin([0,1])][task]
_, cnts = np.unique(label_df, return_counts=True)
loss_weights = (np.ones(num_classes)/cnts)*np.max(cnts)
loss_weights = torch.FloatTensor(loss_weights).to(device)
criterion = nn.CrossEntropyLoss(weight=loss_weights)
print('loss weights:', loss_weights)

loss weights: tensor([1.0364, 1.0000], device='cuda:0')


In [16]:
best_auc_list, best_acc_list, best_auc_acc_list = [], [], []
epochs = 50
iterations = 10
    
if not mtl:
    print('no mtl')
    from train import train
    if channels == 1:
        for i in range(iterations):
            print('Iteration', i)

            from models.Models import SegModel

            esp_model = SegModel(best_model_loc=best_model_loc, 
                                 inp_res = resize_shape, 
                                 num_classes=num_classes, 
                                 channels=4)

            level0_weight = esp_model.espnet.level0.conv.weight[:, 0].unsqueeze(1)
            esp_model.espnet.level0.conv = nn.Conv3d(1, 16, kernel_size=(7, 7, 7), 
                                                     stride=(2, 2, 2), padding=(3, 3, 3), bias=False)

            esp_model.espnet.level0.conv.weight = nn.Parameter(level0_weight)
            esp_model = esp_model.to(device=device)

            optimizer_esp = optim.Adam(esp_model.parameters(), lr=0.0005) # change to adam
        #     exp_scheduler = optim.lr_scheduler.StepLR(optimizer_esp, step_size=7, gamma=0.1)

            exp_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_esp, 
                                                                 mode='min', 
                                                                 factor=0.1, 
                                                                 patience=10, # number of epochs with no change 
                                                                 verbose=True, 
                                                                 threshold=0.0001, 
                                                                 threshold_mode='rel', 
                                                                 cooldown=0, 
                                                                 min_lr=0, 
                                                                 eps=1e-08)


            model, best_wts, best_auc, best_acc, best_auc_acc = train(model=esp_model, 
                               dataloaders=dataloaders,
                               data_transforms=data_transforms,
                               criterion = criterion, 
                               optimizer=optimizer_esp, 
                               scheduler=exp_scheduler,
                               writer=writer,
                               num_epochs=epochs, 
                               verbose=False, 
                               device=device,
                               dataset_sizes=dataset_sizes,
                               channels=1,
                               resize_shape=resize_shape,
                               classes=class_names,
                               volume_val=False,
                               weight_outfile_prefix=model_outfile_dir)
            del esp_model
            del model

            best_auc_list.append(best_auc)
            best_acc_list.append(best_acc)
            best_auc_acc_list.append(best_auc_acc)


            if not os.path.exists('../model_weights/results/'):
                os.makedirs('../model_weights/results/')

            results_outfile_dir = model_outfile_dir + '_epochs-' + str(epochs) +'_iterations-' + str(iterations)
            with open('../model_weights/results/auc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_auc_list, fp)
            with open('../model_weights/results/acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_acc_list, fp)
            with open('../model_weights/results/avg_auc_acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_auc_acc_list, fp)
                
    elif channels == 4:
        criterion = nn.CrossEntropyLoss(weight=loss_weights)
        for i in range(iterations):
            print('Iteration', i)
            from models.Models import SegModel
            esp_model = SegModel(best_model_loc=best_model_loc, inp_res = resize_shape, num_classes=num_classes)

            if channels == 1:
                level0_weight = esp_model.espnet.level0.conv.weight[:, 0].unsqueeze(1)
                esp_model.espnet.level0.conv = nn.Conv3d(1, 16, kernel_size=(7, 7, 7), 
                                                         stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
                esp_model.espnet.level0.conv.weight = nn.Parameter(level0_weight)



            esp_model = esp_model.to(device=device)

            optimizer_esp = optim.Adam(esp_model.parameters(), lr=0.0005) # change to adam
            exp_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_esp, 
                                                                 mode='min', 
                                                                 factor=0.1, 
                                                                 patience=10, # number of epochs with no change 
                                                                 verbose=True, 
                                                                 threshold=0.0001, 
                                                                 threshold_mode='rel', 
                                                                 cooldown=0, 
                                                                 min_lr=0, 
                                                                 eps=1e-08)


            
            model, best_wts, best_auc, best_acc, best_auc_acc = train(model=esp_model, 
                               dataloaders=dataloaders,
                               data_transforms=data_transforms,
                               criterion = criterion, 
                               optimizer=optimizer_esp, 
                               scheduler=exp_scheduler,
                               writer=writer,
                               num_epochs=epochs, 
                               verbose=False, 
                               device=device,
                               dataset_sizes=dataset_sizes,
                               channels=1,
                               resize_shape=resize_shape,
                               classes=class_names,
                               weight_outfile_prefix=model_outfile_dir)
            del esp_model
            del model

            best_auc_list.append(best_auc)
            best_acc_list.append(best_acc)
            best_auc_acc_list.append(best_auc_acc)


            results_outfile_dir = model_outfile_dir + '_epochs-' + str(epochs) +'_iterations-' + str(iterations)

            if not os.path.exists('../model_weights/results/'):
                os.makedirs('../model_weights/results/')

            with open('../model_weights/results/auc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_auc_list, fp)

            with open('../model_weights/results/acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_acc_list, fp)

            with open('../model_weights/results/avg_auc_acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
                pickle.dump(best_auc_acc_list, fp)
            


In [17]:
if mtl:
    print('mtl')
    seg_loss_weight = 1
    surv_loss_weight = 1
    

    brats_seg_ids = glioma_metadata_df[glioma_metadata_df['gt_seg'] == 1].index

    seg_4class_weights = np.load('../data/segmentation_notcropped_4-class_weights.npy')
    seg_4class_weights = torch.FloatTensor(seg_4class_weights).to(device)
    print('seg_4class_weights:', seg_4class_weights)

    seg_2class_weights = np.load('../data/segmentation_notcropped_2-class_weights.npy')
    seg_2class_weights = torch.FloatTensor(seg_2class_weights).to(device)
    print('seg_2class_weights:', seg_2class_weights)
    
    
    for i in range(iterations):
        print('Iteration', i)


        from models.nick_mtl_model import GBMNetMTL
        gbm_net = GBMNetMTL(g_in_features=50, 
                            g_out_features=128, 
                            n_classes=num_classes, 
                            n_volumes=channels, 
                            seg_classes=4, 
                            pretrained=best_model_loc, 
                            class_loss_weights = loss_weights,
                            seg_4class_weights=seg_4class_weights,
                            seg_2class_weights=seg_2class_weights,
                            seg_loss_scale=seg_loss_weight,
                            surv_loss_scale=surv_loss_weight,
                            device = device,
                            brats_seg_ids=brats_seg_ids,
                            standard_unlabled_loss=True,
                            fusion_net_flag=False,
                            modality=modality,
                            take_surv_loss=False)
        gbm_net = gbm_net.to(device)

        optimizer_gbmnet = optim.Adam(gbm_net.parameters(), lr=0.0005) # change to adami
 
        exp_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_gbmnet, 
                                                             mode='min', 
                                                             factor=0.1, 
                                                             patience=10, # number of epochs with no change 
                                                             verbose=True, 
                                                             threshold=0.0001, 
                                                             threshold_mode='rel', 
                                                             cooldown=0, 
                                                             min_lr=0, 
                                                             eps=1e-08)

        from train_mtl import train
        model, best_wts, best_auc, best_acc, best_auc_acc = train(model=gbm_net, 
                       dataloaders=dataloaders,
                       data_transforms=data_transforms,
                       optimizer=optimizer_gbmnet, 
                       scheduler=exp_scheduler,
                       writer=writer,
                       num_epochs=epochs, 
                       verbose=False, 
                       device=device,
                       dataset_sizes=dataset_sizes,
                       channels=channels,
                       classes=class_names,
                       weight_outfile_prefix=model_outfile_dir,
                       pad=0)


        del gbm_net
        del model

        best_auc_list.append(best_auc)
        best_acc_list.append(best_acc)
        best_auc_acc_list.append(best_auc_acc)

        if not os.path.exists('../model_weights/results/'):
            os.makedirs('../model_weights/results/')

        results_outfile_dir = model_outfile_dir + '_epochs-' + str(epochs) +'_iterations-' + str(iterations)
        with open('../model_weights/results/auc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
            pickle.dump(best_auc_list, fp)
        with open('../model_weights/results/acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
            pickle.dump(best_acc_list, fp)
        with open('../model_weights/results/avg_auc_acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
            pickle.dump(best_auc_acc_list, fp)

mtl
seg_4class_weights: tensor([  1.0000, 124.3847,  60.4277, 188.0025], device='cuda:0')
seg_2class_weights: tensor([ 1.0000, 33.4366], device='cuda:0')
Iteration 0
GBMNet!


New Best AUC-acc average:	 0.7248263888888888 	in epoch 0
New Best Dice:	 0.5224299565314197 	in epoch 0
New Best ACC:	 0.6938657407407407 	in epoch 0
New Best AUC-acc average:	 0.8023726851851851 	in epoch 3
New Best ACC:	 0.7795138888888888 	in epoch 3
New Best AUC:	 0.8136574074074074 	in epoch 6
New Best AUC:	 0.8252314814814814 	in epoch 7
New Best AUC:	 0.8472222222222222 	in epoch 11
New Best AUC-acc average:	 0.8029513888888888 	in epoch 12
Epoch    32: reducing learning rate of group 0 to 5.0000e-05.
New Best AUC-acc average:	 0.8125 	in epoch 43
New Best ACC:	 0.8009259259259259 	in epoch 43

Finished Training
Iteration 1
GBMNet!


New Best AUC-acc average:	 0.5619212962962963 	in epoch 0
New Best Dice:	 0.45274225641790894 	in epoch 0
New Best ACC:	 0.5 	in epoch 0
New Best AUC-acc average:	 0.7054398148148149 	in epoch 1
New Best Dice:	 0.45439949146805564 	in epoch 1
New Best ACC:	 0.6898148148148149 	in epoch 1
New Best Dice:	 0.468387501011641 	in epoch 2
New Best AUC-acc average:	 0.752025462962963 	in epoch 5
New Best AUC-acc average:	 0.8153935185185185 	in epoch 6
New Best AUC:	 0.8356481481481481 	in epoch 6
New Best ACC:	 0.7951388888888888 	in epoch 6
New Best AUC-acc average:	 0.8156828703703703 	in epoch 9
New Best AUC:	 0.8518518518518519 	in epoch 9
Epoch    27: reducing learning rate of group 0 to 5.0000e-05.
Epoch    38: reducing learning rate of group 0 to 5.0000e-06.
Epoch    49: reducing learning rate of group 0 to 5.0000e-07.

Finished Training
Iteration 2
GBMNet!


New Best AUC-acc average:	 0.5387731481481481 	in epoch 0
New Best Dice:	 0.5396393722905258 	in epoch 0
New Best ACC:	 0.5381944444444444 	in epoch 0
New Best AUC-acc average:	 0.6412037037037036 	in epoch 2
New Best AUC-acc average:	 0.8003472222222222 	in epoch 3
New Best ACC:	 0.7893518518518519 	in epoch 3
New Best AUC:	 0.6168981481481481 	in epoch 6
New Best AUC:	 0.7060185185185186 	in epoch 7
New Best AUC:	 0.7256944444444444 	in epoch 8
New Best AUC:	 0.8310185185185186 	in epoch 9
New Best AUC-acc average:	 0.8200231481481481 	in epoch 10
New Best AUC:	 0.8506944444444444 	in epoch 10
Epoch    22: reducing learning rate of group 0 to 5.0000e-05.
Epoch    33: reducing learning rate of group 0 to 5.0000e-06.
Epoch    44: reducing learning rate of group 0 to 5.0000e-07.

Finished Training
Iteration 3
GBMNet!


KeyboardInterrupt: 

In [1]:
!nvidia-smi

Wed Jan  6 16:31:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 47%   65C    P2   183W / 250W |   7539MiB / 11019MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
7539+3761

11300

# need to combine
- nick_mtl_model
- train_mtl
- joint_model2

In [ ]:
pecify align_corners=True

In [ ]:
train_mtl.py:130

In [ ]:
nn.functional.upsample -> nn.functional.interpolate

### 